# Digital Image Processing EC9570
## Garbage Classification project
2021/E/045
2021/E/179

## 1. Imports & Setup

In [10]:
import random
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import torch

## 2. Inspecting the Dataset

In [12]:
data_dir='./raw/'

In [13]:
import os
def class_count(directory):
    image_directories=[]
    for folder in os.listdir(directory):
        sub_folder_path=os.path.join(data_dir,folder)
        image_directories.append(sub_folder_path)
        print(f"{folder} - {len(os.listdir(sub_folder_path))}")


## 3. Data Augmentation Functions

#### Rotation

In [ ]:

def rotate(image):

    angle=random.randint(-30,30)
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rot_matrix, (w, h))

    return rotated_image


### Flip

In [ ]:
def flip(image):
    flip_code=random.choice([0,1,-1])
    flipped_image = cv2.flip(image, flip_code)
    return flipped_image

### Random Erasing

In [ ]:
# Randomly selects a rectangle region in an image and erases its pixels.
def random_erasing(image):
    erased_image = image.copy()
    (h, w) = image.shape[:2]
    # Define a random rectangle area to erase
    erase_h = random.randint(int(h * 0.1), int(h * 0.3))
    erase_w = random.randint(int(w * 0.1), int(w * 0.3))
    x1 = random.randint(0, w - erase_w)
    y1 = random.randint(0, h - erase_h)
    # Set the area to black
    erased_image[y1:y1 + erase_h, x1:x1 + erase_w] = (0, 0, 0)
    return erased_image


## 4. Balancing the Dataset

In [20]:
import os
import cv2
import random

source_dir = './raw/'
output_dir = './processed/'

# List of available augmentation functions
augmentations = [rotate, flip, random_erasing]

# 1. Calculate class sizes and find the target number of images
class_counts = {}
for folder in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder)
    if os.path.isdir(folder_path):
        class_counts[folder] = len(os.listdir(folder_path))

if not class_counts:
    print("Source directory is empty or contains no subdirectories.")
else:
    max_images = max(class_counts.values())
    print(f"Target image count per class: {max_images}")

    # 2. Create output directories and process each class
    for class_name, current_count in class_counts.items():
        print(f"Processing class: {class_name}")

        # Create corresponding output directory
        output_class_dir = os.path.join(output_dir, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        source_class_dir = os.path.join(source_dir, class_name)
        image_files = [f for f in os.listdir(source_class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Copy original images
        for filename in image_files:
            img_path = os.path.join(source_class_dir, filename)
            image = cv2.imread(img_path)
            if image is None:
                continue
            save_path = os.path.join(output_class_dir, filename)
            cv2.imwrite(save_path, image)

        #  Generate new images if needed
        num_to_generate = max_images - current_count
        print(f"Generating {num_to_generate} new images for {class_name}...")

        for i in range(num_to_generate):
            # Pick a random original image to augment
            random_image_name = random.choice(image_files)
            image_path = os.path.join(source_class_dir, random_image_name)
            image = cv2.imread(image_path)

            if image is None:
                continue

            # Apply a random number of augmentations
            num_aug_to_apply = random.randint(1, len(augmentations))
            augs_to_apply = random.sample(augmentations, num_aug_to_apply)

            augmented_image = image
            for aug_func in augs_to_apply:
                augmented_image = aug_func(augmented_image)

            # Save the new image
            base_filename, file_ext = os.path.splitext(random_image_name)
            new_filename = f"{base_filename}_aug_{i}{file_ext}"
            save_path = os.path.join(output_class_dir, new_filename)
            cv2.imwrite(save_path, augmented_image)

    print("\nDataset balancing complete.")


Target image count per class: 5325
Processing class: battery
Generating 4380 new images for battery...
Processing class: biological
Generating 4340 new images for biological...
Processing class: brown-glass
Generating 4718 new images for brown-glass...
Processing class: cardboard
Generating 4434 new images for cardboard...
Processing class: clothes
Generating 0 new images for clothes...
Processing class: green-glass
Generating 4696 new images for green-glass...
Processing class: metal
Generating 4556 new images for metal...
Processing class: paper
Generating 4275 new images for paper...
Processing class: plastic
Generating 4460 new images for plastic...
Processing class: shoes
Generating 3348 new images for shoes...
Processing class: trash
Generating 4628 new images for trash...
Processing class: white-glass
Generating 4550 new images for white-glass...

Dataset balancing complete.


## Pre-Processing Stratey

now we have a balanced dataset with each class containing the same number of images. The images have been resized to 128x128 pixels, and various augmentations have been applied to generate new images for classes that initially had fewer samples. Additionally, histogram equalization has been applied to enhance image contrast.
1. Resize to 128x128
2. Denoise (Gaussian blur)
3. Sharpen
4. Histogram equalization
5. Normalize to [0, 1]

#### Resize

In [30]:
# Resize to 128x128
def resize(image, size=(128, 128)):
    resized_img = cv2.resize(image, size)
    return resized_img

#### Denoise

In [31]:
def denoise(image):
    h, w = image.shape[:2]
    # Kernel size: 1/50th of the smallest dimension, rounded to nearest odd integer
    ksize = max(3, min(15, (min(h, w) // 50) | 1))
    blurred_image = cv2.GaussianBlur(image, (ksize, ksize), 0)
    return blurred_image


#### Sharpen

In [32]:
def sharpen(image):
    h, w = image.shape[:2]
    # Sharpening strength: scale with image size, but keep reasonable bounds
    alpha = min(2.0, max(1.0, min(h, w) / 128))
    # Sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1, 4 * alpha + 1, -1],
                       [0, -1, 0]])
    kernel = kernel / (4 * alpha)
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened


#### Histogram Equilization

In [33]:
def histogram_equalization(image):
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    # Equalize the histogram of the Y channel
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    # Convert back to BGR
    equalized_img = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
    return equalized_img

#### Normalize

In [34]:
def normalize(image):
    # Convert to float32 and scale to [0, 1]
    norm_img = image.astype(np.float32) / 255.0
    return norm_img


#### Pre Processing the image dataset


In [ ]:
import os
import cv2
import numpy as np

source_dir = './processed/'
output_dir = './final/'

for directory in os.listdir(source_dir):
    source_class_dir = os.path.join(source_dir, directory)
    output_class_dir = os.path.join(output_dir, directory)
    os.makedirs(output_class_dir, exist_ok=True)

    for filename in os.listdir(source_class_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(source_class_dir, filename)
            image = cv2.imread(img_path)
            if image is None:
                continue

            # Apply preprocessing steps
            image = resize(image)
            image = denoise(image)
            image = sharpen(image)
            image = histogram_equalization(image)
            image = normalize(image)

            # Save the preprocessed image
            save_path = os.path.join(output_class_dir, filename)
            cv2.imwrite(save_path, (image * 255).astype(np.uint8))
